In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

In [2]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data

    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    # Return
    return df_clientes_activos

In [3]:
vars_to_scale = ["MONTO_ULT_FACTURA",
                 "MORA_1_30",
                 "MORA_31_60",
                 "MORA_61_90",
                 "MORA_91_120",
                 "MORA_121_150",
                 "MORA_151_180",
                 "MORA_180",
                 "TOT_AMT_OVERDUE"]

id_cols = ["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]

past_periods = [7, 28, 84, 168]

In [4]:
def create_master_mora(mora: SQLPartitionedDataSet,
                       cliente_activo: pd.DataFrame,
                       parameters: Dict,
                       date: str) -> pd.DataFrame:
    """Creates master table with customer default features for one period of data
    Parameters
    ----------
    mora: 
        dataset defined in ``catalog_raw.yml`` with raw data information related to customers in default
    cliente_activo: 
        dataset defined in ``catalog_raw.yml`` with raw data information related to active clients at EoP
    date: 
        period to process
    parameters: 
        set of project parameters defined in ``parameters.yml``
    Returns
    -------
    pd.DataFrame
        Master table with customer default features for one period
        
    """

    # Initialize logger
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "mora"
    overwrite = parameters["masters"][table_name]["overwrite"]

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0 and overwrite is False:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_morosidad_activo_past = pd.read_parquet(match[0], engine="pyarrow")

    else:
        log.info("Creating master table mora")

        # Read parameters
        look_back_days = parameters["masters"]["mora"]["look_back_days"]
        start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")

        # Calculate period to load for active clients
        log.info("Reading tables...")
        period_to_load = get_previous_month(date)

        df_clientes = cliente_activo[["CUSTOMER_ID"]]

        # Get intencion morosidad table
        df_morosidad = mora.filter_by(date=[start_date, date]).drop_duplicates()

        df_morosidad["FECHA_PAGO"] = pd.to_datetime(df_morosidad["PAGO"].astype(str))
        df_morosidad["EDAD_MORA"] = (df_morosidad["FECHA"] - df_morosidad["FECHA_PAGO"]) / np.timedelta64(1, "D")

        # Format
        df_morosidad["FECHA"] = df_morosidad["FECHA"].dt.strftime("%Y%m%d")

        # Merging
        log.info("Merging with EOP..")
        df_morosidad_activo = df_clientes.merge(df_morosidad,
                                                on="CUSTOMER_ID",
                                                how="inner")

        # Create scale variables
        log.info("Scaling numeric variables...")
        for var in vars_to_scale:
            log.info(f"---- {var}")
            df_morosidad_activo[var + "_scaled"] = scale_values(df=df_morosidad_activo,
                                                                vars_to_groupby=["DATE_EXP"],
                                                                var_to_scale=var,
                                                                by_ranking=False)

        create_evolution_variables(df=df_morosidad_activo,
                                   var_name='PROP_OVERDUE',
                                   numerator='TOT_AMT_OVERDUE',
                                   denominator='MONTO_ULT_FACTURA')

        # Creating number of bills in default
        df_morosidad_activo["PAGO"] = df_morosidad_activo["PAGO"].astype(str)
        df_bills = df_morosidad_activo[["CUSTOMER_ID", "PAGO", ]].drop_duplicates()
        df_bills["FACTURAS"] = 1
        df_bills = add_relative_calculate_past(df=df_bills,
                                               id_cols=["CUSTOMER_ID"],
                                               periods=past_periods,
                                               agg={'FACTURAS': [np.size]},
                                               date_col="PAGO",
                                               start_date=start_date,
                                               end_date=date,
                                               period_freq="D")

        # Create expanded variables
        log.info("Creating past variables...")
        df_morosidad_activo_past = add_relative_calculate_past(df=df_morosidad_activo,
                                                               id_cols=["CUSTOMER_ID"],
                                                               periods=past_periods,
                                                               agg={
                                                                   "TOT_AMT_OVERDUE_scaled": [np.nanmax],
                                                                   "PROP_OVERDUE": [np.nanmax],
                                                                   "EDAD_MORA": [np.nanmax],
                                                                   "MORA_1_30_scaled": [np.nanmax],
                                                                   "MORA_31_60_scaled": [np.nanmax],
                                                                   "MORA_61_90_scaled": [np.nanmax],
                                                                   "MORA_91_120_scaled": [np.nanmax],
                                                                   "MORA_121_150_scaled": [np.nanmax],
                                                                   "MORA_151_180_scaled": [np.nanmax],
                                                                   "MORA_180_scaled": [np.nanmax]},
                                                               date_col="FECHA",
                                                               start_date=start_date,
                                                               end_date=date,
                                                               period_freq="D")

        df_morosidad_activo_past = df_morosidad_activo_past.merge(df_bills, on=["CUSTOMER_ID"])

        # Add date variables
        df_morosidad_activo_past["DATE_EXP"] = period_to_load
        df_morosidad_activo_past["DATE_CALC"] = date

        # Change variable names 
        table_preffix = parameters["masters"]["mora"]["table_preffix"]
        rename_table(df_morosidad_activo_past,
                     preffix=table_preffix,
                     ids_to_exclude=id_cols)
        log.info(f"Exporting {df_morosidad_activo_past.shape[0]} rows and {df_morosidad_activo_past.shape[1]} columns")

        if write_to_parquet:
            log.info(f"\n------ Writing {table_name} for period {date} to parquet ------")
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date}.parquet"
            df_morosidad_activo_past.to_parquet(file, engine="pyarrow")

        log.info(f"Exporting {df_morosidad_activo_past.shape[0]} rows and {df_morosidad_activo_past.shape[1]} columns")

    return df_morosidad_activo_past

In [5]:
cliente_activo=catalog.load("cliente_activo")
mora=catalog.load("mora")

2020-12-30 19:49:52,536 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...
2020-12-30 19:49:52,538 - kedro.io.data_catalog - INFO - Loading data from `mora` (SQLPartitionedDataSet)...


In [6]:
dates = calculate_dates_to_process_for_master(parameters, table_name="mora")
print(dates)

['20181203', '20181231', '20190128', '20190225', '20190325', '20190422']


In [7]:
for date in dates:
    print(f"Processing date {date}")
    cliente_activo_df= create_cliente_activo(cliente_activo,date)
    create_master_mora(mora,cliente_activo_df,parameters,date)
    

Processing date 20181203
2020-12-30 19:49:54,848 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_uy_customer_status where UPPER(STATUS) LIKE '%ACTIVO%' and DATE_EXP = 201811


/u01/miniconda3/envs/uypo37/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


2020-12-30 19:49:57,695 - aa_engine_pkg.assets.utils.utilities - INFO - Creating master table mora
2020-12-30 19:49:57,697 - aa_engine_pkg.assets.utils.utilities - INFO - Reading tables...
select CUSTOMER_ID, FECHA, MONTO_ULT_FACTURA, PAGO, MORA_1_30, MORA_31_60, MORA_61_90, MORA_91_120, MORA_121_150, MORA_151_180, MORA_180, TOT_AMT_OVERDUE, DATE_EXP from stg_uy_cliente_moroso where FECHA >= to_date('20180910', 'yyyymmdd') and FECHA < to_date('20181203', 'yyyymmdd')
2020-12-30 19:50:59,083 - aa_engine_pkg.assets.utils.utilities - INFO - Merging with EOP..
2020-12-30 19:51:00,844 - aa_engine_pkg.assets.utils.utilities - INFO - Scaling numeric variables...
2020-12-30 19:51:00,845 - aa_engine_pkg.assets.utils.utilities - INFO - ---- MONTO_ULT_FACTURA
2020-12-30 19:51:02,088 - aa_engine_pkg.assets.utils.utilities - INFO - ---- MORA_1_30
2020-12-30 19:51:02,891 - aa_engine_pkg.assets.utils.utilities - INFO - ---- MORA_31_60
2020-12-30 19:51:03,306 - numexpr.utils - INFO - Note: NumExpr dete